In [ ]:
from dandi.dandiapi import DandiAPIClient

dandiset_id = '000041'  # ephys dataset from the Svoboda Lab
filepath = 'sub-BWRat17/sub-BWRat17_ses-BWRat17-121712_ecephys.nwb'  # 1.5GB file
with DandiAPIClient() as client:
    asset = client.get_dandiset(dandiset_id, 'draft').get_asset_by_path(filepath)
    s3_url = asset.get_content_url(follow_redirects=1, strip_query=True)

In [ ]:
dandiset_id, filepath="0000041","sub-BWRat17/sub-BWRat17_ses-BWRat17-121712_ecephys.nwb"
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

io = NWBHDF5IO(s3_url,mode='r',load_namespaces=True,driver='ros3')
nwb = io.read()


In [ ]:
nwb2widget(nwb)

In [ ]:
nwb.units.to_dataframe()

In [ ]:
import numpy

spiketrain=numpy.zeros(len(nwb.units["spike_times"]))
outputspiketrain=numpy.zeros(len(nwb.units["spike_times"]))


for i in range(len(nwb.units["spike_times"])):
    #print(i)
    spiketrain[i] = numpy.histogram(nwb.units["spike_times"][i], 
                         bins = 1, 
                         range = (0, 2))[0]/2
    
    outputspiketrain[i] = numpy.histogram(nwb.units["spike_times"][i], 
                         bins = 1, 
                         range = (2, 4))[0]/2


# select only excitatory neurons & firing rate not equal to zero   

spiketrainN=spiketrain[(nwb.units["cell_type"][:]=="excitatory") & ((spiketrain>0) | (outputspiketrain>0))]
outputspiketrainN=outputspiketrain[(nwb.units["cell_type"][:]=="excitatory") & ((spiketrain>0) | (outputspiketrain>0))]

  